In [5]:
import nbimporter
from dice_rollers import rollSixStats, findStatModifier, rollxdy

In [26]:
class Character(object):
    def __init__(self, name: str, hit_die: str = "1d10"):
        self.name = name
        self.hit_die = hit_die
        self.state = "alive"
        self.armorClass = 15

        # set attributes
        self.str = self.dex = self.con = self.int = self.wis = self.cha = 0

        self.get_stat_mod = get_stat_mods  # I don't actually know if this is good practice

    def setStats(self, stat_to_modify: str, stat_number: int):
        setattr(self, stat_to_modify, stat_number)

    def setHitpoints(self):
        # first level shouldn't have a roll
        self.hitpoints = rollxdy(self.hit_die) + self.con_mod

    def getAttacked(self, attack_dictionary: dict):
        print("%s gets attacked" % self.name)
        if attack_dictionary["to_hit"] >= self.armorClass:
            print("it's a hit")
            self.takeDamage(attack_dictionary["damage"])
        else:
            print("it's a miss")

    def takeDamage(self, damage: int):
        if damage >= self.hitpoints:
            self.state = "dead"
        self.hitpoints -= damage

    def equipWeapon(self, weapon: Weapon):
        self.weapon = weapon

    def attack(self, attack_stat: int = "str"):
        modifier = sself.get_stat_mod(getattr(self, attack_stat))

        roll = rollxdy("1d20")  # I'd recommend changing rollxdy to accept 2 ints instead of a string

        to_hit = roll + modifier

        if roll == 20:
            damage = self.weapon.dealDamage(is_crit=True)
        elif roll != 20:
            damage = self.weapon.dealDamage(is_crit=False)
        else:  # You never hit this else statement!
            damage = 0

        damage += modifier

        return {"to_hit": to_hit, "damage": damage}

In [7]:
class Weapon(object):
    def __init__(
        self, damage_die: str = "1d6", statistic: str = "str", modifier: int = 0
    ):
        self.damage_die = damage_die
        self.statistic = statistic
        self.modifier = modifier

    def dealDamage(self, is_crit: bool = False):
        if is_crit:
            return rollxdy(self.damage_die) + rollxdy(self.damage_die) + self.modifier
        else:
            return rollxdy(self.damage_die) + self.modifier

In [8]:
def createCharacterWithStatList(name: str, stat_list: list, stat_rank_order: list):  # camelCase is for classes, use snake_case
    """
    Creates a Character with given name.
    Stats are taken from stat_list and assigned
    in descending rank order based on stat_rank_order
    """
    stat_list.sort(reverse=True)
    character = Character(name)
    for i in range(len(stat_rank_order)):
        character.setStats(stat_to_modify=stat_rank_order[i], stat_number=stat_list[i])
    character.setModifiers()
    character.setHitpoints()
    return character

In [27]:
stat_list = rollSixStats()
stat_rank_order = ["str", "dex", "con", "int", "wis", "cha"]
Dan = createCharacterWithStatList("Dan", stat_list, stat_rank_order)
Dan.setHitpoints()
katana = Weapon()
Dan.equipWeapon(katana)
Dan.attack()

{'to_hit': 7, 'damage': 13}